In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from   tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# Load data
data = pd.read_csv('raw_data.csv')
print("First few rows of raw data:")
print(data.head())

# Filter out rows with synthetic thefts
cleaned_data = data[data['theft'] == 'Normal']

# Drop the 'theft' column (optional)
cleaned_data = cleaned_data.drop(columns=['theft'])
print("\nColumn Names in cleaned_data:")
print(cleaned_data.columns)

# Aggregate energy consumption into hourly totals
cleaned_data['Total_Electricity'] = cleaned_data[
    ['Electricity:Facility [kW](Hourly)', 'Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)', 
     'Heating:Electricity [kW](Hourly)', 'InteriorLights:Electricity [kW](Hourly)']
].sum(axis=1)

cleaned_data['Total_Gas'] = cleaned_data[
    ['Gas:Facility [kW](Hourly)', 'Heating:Gas [kW](Hourly)', 'InteriorEquipment:Gas [kW](Hourly)', 
     'Water Heater:WaterSystems:Gas [kW](Hourly)']
].sum(axis=1)

print("\nAggregated Energy Consumption:")
print(cleaned_data[['Total_Electricity', 'Total_Gas']].head())

# Convert timestamp and extract day of week/weekend flag
cleaned_data['timestamp'] = pd.to_datetime(cleaned_data['timestamp'])
cleaned_data['DayOfWeek'] = cleaned_data['timestamp'].dt.dayofweek
cleaned_data['IsWeekend'] = cleaned_data['DayOfWeek'].isin([5, 6]).astype(int)

# Normalize energy consumption columns
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = cleaned_data.copy()
normalized_data[['Total_Electricity', 'Total_Gas']] = scaler.fit_transform(
    cleaned_data[['Total_Electricity', 'Total_Gas']]
)

print("\nNormalized Total Electricity and Gas:")
print(normalized_data[['Total_Electricity', 'Total_Gas']].head())

# Split data into train, validation, and test sets
train_data, test_data = train_test_split(normalized_data, test_size=1/5, shuffle=False)
train_data, val_data = train_test_split(train_data, test_size=0.1, shuffle=False)

print("\nTrain Data Shape:", train_data.shape)
print("Validation Data Shape:", val_data.shape)
print("Test Data Shape:", test_data.shape)

# Create input features for the MLP model
def create_input_features(data):
    X, y = [], []
    for i in range(len(data) - 24):  # Use a sliding window of 24 hours
        X.append(data.iloc[i:i+24][['Total_Electricity', 'Total_Gas', 'DayOfWeek', 'IsWeekend']].values)
        y.append(data.iloc[i+24]['Total_Electricity'])  # Predict next hour's electricity consumption
    return np.array(X), np.array(y)

X_train, y_train = create_input_features(train_data)
X_val, y_val = create_input_features(val_data)
X_test, y_test = create_input_features(test_data)

print("\nX_train Shape:", X_train.shape)
print("y_train Shape:", y_train.shape)

# Build the MLP model
def build_mlp_model(input_shape):
    model = Sequential([
        Dense(30, activation='relu', input_shape=input_shape),  # Hidden layer with 30 neurons
        Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer=SGD(learning_rate=0.01), loss='mse')
    return model

model = build_mlp_model(input_shape=(X_train.shape[1], X_train.shape[2]))
print("\nModel Summary:")
model.summary()

First few rows of raw data:
   0  Electricity:Facility [kW](Hourly)  Fans:Electricity [kW](Hourly)  \
0  0                          22.035977                       3.586221   
1  1                          14.649757                       0.000000   
2  2                          14.669567                       0.000000   
3  3                          14.677808                       0.000000   
4  4                          14.824794                       0.000000   

   Cooling:Electricity [kW](Hourly)  Heating:Electricity [kW](Hourly)  \
0                               0.0                               0.0   
1                               0.0                               0.0   
2                               0.0                               0.0   
3                               0.0                               0.0   
4                               0.0                               0.0   

   InteriorLights:Electricity [kW](Hourly)  \
0                                 4.589925

C:\Users\rionv\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 24, 30)         │           150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24, 1)          │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 181 (724.00 B)

 Trainable params: 181 (724.00 B)

 Non-trainable params: 0 (0.00 B)